In [93]:
import numpy as np 
import pandas as pd
import matplotlib as plt
import math
import re
from collections import Counter
from sklearn.decomposition import TruncatedSVD


In [94]:
credits=pd.read_csv(r'F:\NLP project\tmdb_5000_credits.csv\tmdb_5000_credits.csv',encoding_errors = 'ignore')
movies=pd.read_csv(r'F:\NLP project\tmdb_5000_movies.csv\tmdb_5000_movies.csv',encoding_errors = 'ignore')
credits_column_renamed=credits.rename(columns={"movie_id":"id"})
movies_df_merged=movies.merge(credits_column_renamed,on='id')
movies_clean=movies_df_merged[['id','original_title','overview','label']]

In [95]:
col=movies_clean.columns
res=[]
for c in col:
    d=movies_clean[col].values.tolist()
    res.append(d)

In [103]:
def tf(word, count):
    return count[word] / sum(count.values())


def n_containing(word, count_list):
    return sum(1 for count in count_list if word in count)
 
 
def idf(word, count_list):
    return math.log(len(count_list) / (1 + n_containing(word, count_list)))


def tfidf(word, count, count_list):
    return tf(word, count) * idf(word, count_list)


def as_num(x):
    y='{:.5f}'.format(x)
    return(y)


def cosine_similarity(x,y):
    num = x.dot(y.T)
    denom = np.linalg.norm(x) * np.linalg.norm(y)
    cos = num/ denom
    return cos

def euclideandistance(x,y):
    return np.sqrt(np.sum(np.square(x-y)))


In [5]:
stopwords = []
movie_tfidfvector=[]
with open("stopword.txt", "r") as f:
    while True:
        line = f.readline()
        if not line:
            break
        stopwords.append(line.strip("\n").strip())
countlist = []
for i in range(0,1470):
    with open(r"F:\NLP project\movie/"+str(i)+".txt", "r",encoding='utf-8') as f:    
        data = f.read()   
        
    text = re.sub(r"'s", " 's", data)
    text = re.sub(r"'ve", " 've", data)
    text = re.sub(r"n't", " n't", data)
    text = re.sub(r"'re", " 're", data)
    text = re.sub(r"'d", " 'd", data)
    text = re.sub(r"'ll", " 'll", data)
    text = re.sub(r"[^A-Za-z]", " ", data)
    do_lower_case=True
    if do_lower_case:
        text=text.strip().lower()
    tokens = text.split()
    text_words = [token for token in tokens if token not in stopwords]
    count = Counter(text_words)
    countlist.append(count)
    docvector=np.array([tfidf(word, count, countlist) for word in count])
    docvector1=np.reshape(docvector,(1,len(docvector)))
    docvector2 = abs(np.sort(-docvector1))
    movie_tfidfvector.append(docvector2[0][:5])

In [97]:
indices=pd.Series(movies_clean.index,index=movies_clean.original_title)

In [98]:
vector=movie_tfidfvector[4]
vector_new=vector.reshape(1,5)
movie_tfidfvector_linshi=np.array(movie_tfidfvector)
movie_tfidfvector_new=movie_tfidfvector_linshi.T
movie_tfidfvector_new.shape

(5, 1470)

In [99]:
svd = TruncatedSVD(n_components = 3)
svdMatrix = svd.fit_transform(movie_tfidfvector_linshi)
svdMatrix[3]

array([ 0.07216203, -0.0151446 , -0.0027677 ])

In [100]:
def recommend(a):
    movie_tfidfvector_linshi=np.array(movie_tfidfvector)
    svd = TruncatedSVD(n_components = 3)
    svdMatrix = svd.fit_transform(movie_tfidfvector_linshi)
    targetmov=svdMatrix[a]
    score=[]
    for i,j in enumerate(svdMatrix):
        score.append([i,euclideandistance(j,targetmov)])
    Relatedmovie=sorted(score,key = lambda x: x[1],reverse = False)   
    res=Relatedmovie[1:11]
    movies_indices = [i[0] for i in res]
    series=movies_clean.label.iloc[movies_indices]
    return series.values

In [101]:
a=0
acclis=[]
for i in range(0,100):
    accuracy=0
    a=0
    for j in recommend(i):
        if j==movies_clean['label'][i]:
            a=a+1
    accuracy=a/10
    acclis.append(accuracy)

In [102]:
acc=sum(acclis)/len(acclis)
print('%.3f' % acc)

0.472
